In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPool2D, AveragePooling2D,Dense, Flatten, Input, Dropout
from tensorflow.keras.models import Sequential, Model
import autokeras as ak

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
datasize = len(y_train)
x_val, y_val = x_train[int(0.10*datasize):int(0.25*datasize)], y_train[int(0.10*datasize):int(0.25*datasize)]
print(type(x_train), type(y_train))
print(x_val.shape, y_val.shape)

11493376/11490434 [==============================] - 3s 0us/step

In [3]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=3)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=20)

# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)

# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))



Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
image_block_1/b...|vanilla           |?                 
image_block_1/n...|True              |?                 
image_block_1/a...|False             |?                 
image_block_1/c...|3                 |?                 
image_block_1/c...|1                 |?                 
image_block_1/c...|2                 |?                 
image_block_1/c...|True              |?                 
image_block_1/c...|False             |?                 
image_block_1/c...|0.25              |?                 
image_block_1/c...|32                |?                 
image_block_1/c...|64                |?                 
classification_...|flatten           |?                 
classification_...|0.5               |?                 
optimizer         |adam              |?                 
learning_rate     |0.001             |?                 

Epoch 1/20


InternalError:  dnn PoolBackward launch failed
	 [[node gradient_tape/model/max_pooling2d/MaxPool/MaxPoolGrad (defined at C:\Users\alien\AppData\Roaming\Python\Python38\site-packages\autokeras\utils\utils.py:88) ]] [Op:__inference_train_function_11242]

Function call stack:
train_function


In [ ]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=1)
clf.fit(x_train, y_train, validation_split=0.15, epochs=10)

predicted_y = clf.predict(x_test)
print(predicted_y)

print(clf.evaluate(x_test, y_test))


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
image_block_1/r...|False             |?                 
image_block_1/r...|resnet50          |?                 
image_block_1/r...|False             |?                 
classification_...|flatten           |?                 
classification_...|0                 |?                 
optimizer         |adam              |?                 
learning_rate     |0.001             |?                 

Epoch 1/10
1248/1594 [======================>.......] - ETA: 14s - loss: 0.4309 - accuracy: 0.8987

KeyboardInterrupt: ignored

In [ ]:
img_height, img_width = x_train[0].shape
xIn = Input((img_height,img_width))

x = Flatten()(xIn)
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(x)
x = Dense(128, activation = 'relu')(x)
x = Dense(56, activation = 'relu')(x)
x = Dense(28, activation = 'relu')(x)
xOut = Dense(10, activation = 'softmax')(x)

model = Model(xIn, xOut)
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['acc'])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
rescaling_2 (Rescaling)      (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_9 (Dense)              (None, 56)                7224      
_________________________________________________________________
dense_10 (Dense)             (None, 28)                1596      
_________________________________________________________________
dense_11 (Dense)             (None, 10)                290 

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
      './best_model',
      monitor='val_loss',
      save_best_only = True
  ), 
    tf.keras.callbacks.EarlyStopping(
        min_delta=0.002,
        monitor='val_acc',
        patience=10
    )]

model.fit(x_train, y_train, epochs=10, initial_epoch=0, validation_data=(x_val, y_val), callbacks=callbacks)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


1875/1875 [==============================] - 5s 2ms/step - loss: 0.2484 - acc: 0.9258 - val_loss: 0.1071 - val_acc: 0.9664
INFO:tensorflow:Assets written to: ./best_model/assets
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1023 - acc: 0.9686 - val_loss: 0.0789 - val_acc: 0.9756
INFO:tensorflow:Assets written to: ./best_model/assets
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0745 - acc: 0.9770 - val_loss: 0.0504 - val_acc: 0.9841
INFO:tensorflow:Assets written to: ./best_model/assets
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0582 - acc: 0.9818 - val_loss: 0.0496 - val_acc: 0.9843
INFO:tensorflow:Assets written to: ./best_model/assets
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0464 - acc: 0.9855 - val_loss: 0.0440 - val_acc: 0.9864
INFO:tensorflow:Assets written to: ./best_model/assets
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/ste

In [ ]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1041 - acc: 0.9752


[0.10408436506986618, 0.9751999974250793]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/fashion_mnist.zip /content
!unzip '/content/fashion_mnist.zip' -d'/content/fashion_mnist'

Streaming output truncated to the last 5000 lines.
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/58411.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/6837.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/62424.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/42681.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/385.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/37714.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/37960.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/61416.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/69983.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/6845.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/43698.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/65572.jpg  
 extracting: /content/fashion_mnist/fashion_mnist/Trouser/26354.jpg  
 extracting: /content/fashion_mnist/fashion

In [ ]:
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.utils
from tensorflow.keras import layers
import os, cv2, random
import matplotlib.pyplot as plt
import numpy as np

label_text = ['Ankle Boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'T-Shirt', 'Trouser']
label_dict = {
    'Ankle Boot' : 0,
    'Bag' : 1,
    'Coat' : 2,
    'Dress' : 3,
    'Pullover' : 4,
    'Sandal' : 5,
    'Shirt' : 6,
    'Sneaker' : 7,
    'T-Shirt' : 8,
    'Trouser' : 9
}# create dataset split
datapath = './fashion_mnist/fashion_mnist'
x, y = [], []
for folder in os.listdir(datapath):
    folder_path = datapath + '/' + folder
    class_idx = label_dict[folder]
    for img in os.listdir(folder_path):
        image = cv2.imread(folder_path + '/' + img, 0)
        x.append(image)
        y.append(class_idx)

# shuffle and split
combined = list(zip(x, y))

random.shuffle(combined)

x, y = zip(*combined)
datasize = len(y)
x_train, y_train = x[:int(0.60*datasize)], y[:int(0.60*datasize)]
x_val, y_val = x[int(0.60*datasize):int(0.80*datasize)], y[int(0.60*datasize):int(0.80*datasize)]
x_test, y_test = x[int(0.80*datasize):], y[int(0.80*datasize):]

In [ ]:
import numpy as np
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_val = np.asarray(x_val)
y_val = np.asarray(y_val)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [ ]:
clf = ak.ImageClassifier(overwrite=True, max_trials=20)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=10)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 1 Complete [00h 01m 51s]
val_loss: 0.24511945247650146

Best val_loss So Far: 0.24511945247650146
Total elapsed time: 00h 01m 51s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
image_block_1/b...|resnet            |vanilla           
image_block_1/n...|True              |True              
image_block_1/a...|True              |False             
image_block_1/i...|True              |None              
image_block_1/i...|True              |None              
image_block_1/i...|0                 |None              
image_block_1/i...|0                 |None              
image_block_1/i...|0.1               |None              
image_block_1/i...|0                 |None              
image_block_1/r...|False             |None              
image_block_1/r...|resnet50          |None              
image_block_1/r...|True              |None              
classification_...|global_avg        |flatten           
classification_...|0                 |0